In [1]:
# Intialization
import os
import sys
import glob
import shutil

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
#1. Create spark session obj
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("export_fraud_detection_dataMart").config("spark.driver.memory", "8g").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
print("Loading Gold layer data mart tables from Hive...")

#Load the tables directly from the database
# We use the standard 'database_name.table_name' format
df = spark.table("financial_db.fraud_detection_gold")

# 3. Quick Verification
print(f"datamart rows Count: {df.count()}")

df.printSchema()

Loading Gold layer data mart tables from Hive...
datamart rows Count: 24386900
root
 |-- transaction_id: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- card: integer (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- label_is_fraud: integer (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_category: integer (nullable = true)
 |-- error_code: string (nullable = true)
 |-- credit_utilization: double (nullable = true)
 |-- state_mismatch: integer (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- yearly_income_person: decimal(10,2) (nullable = true)
 |-- fico_score: integer (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_brand: string (nullable = true)



In [8]:
#2. Define Output Paths (Local Linux Filesystem)
#Using 'file://' to force Spark to write to local disk instead of HDFS

temp_output_folder = 'file:///home/talentum/projectMaster/dataStaging/temp_orc_folder'
final_output_file = '/home/talentum/projectMaster/dataStaging/fraud_mart_final.orc'

#Clean up previous runs if they exist
if os.path.exists(final_output_file):
    os.remove(final_output_file)

os.makedirs(temp_output_folder, exist_ok = True)
    
#3. Coalesce and write
#coalesce(1) forces all data into a single partition
print("Writing single ORC file...")
df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .orc(temp_output_folder)

#4. Rename the output folder
local_folder_path = '/home/talentum/projectMaster/dataStaging/temp_orc_folder'
list_of_files = glob.glob(f"{local_folder_path}/part-*.orc")

if list_of_files:
    actual_file = list_of_files[0]
    #Move and Rename
    shutil.move(actual_file, final_output_file)
    shutil.rmtree(local_folder_path)
    print(f"Success! Single file is ready at: {final_output_file}")
else:
    print('Error: Could not find the generated ORC file.')

Writing single ORC file...
Success! Single file is ready at: /home/talentum/projectMaster/dataStaging/fraud_mart_final.orc
